In [65]:
"""Bokeh Visualization Template

This template is a general outline for turning your data into a 
visualization using Bokeh.
"""
# Data handling
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import glob, os
from datetime import date
import datetime


# Bokeh libraries
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
from bokeh.io import output_file
# Bokeh Libraries
from bokeh.models import ColumnDataSource, CategoricalColorMapper, Div, RangeTool, Range1d, CustomJS, DateRangeSlider
from bokeh.sampledata.stocks import AAPL
from bokeh.plotting import figure
from bokeh.resources import CDN
from bokeh.embed import file_html

def to_integer(dt_time):
    return 10000*dt_time.year + 100*dt_time.month + dt_time.day


In [66]:
import pandas as pd
import numpy as np
import glob, os

# Get CSV files list from a folder
path = 'data/sales_data'
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file) for file in csv_files)

# Concatenate all DataFrames
df   = pd.concat(df_list, ignore_index=True)
df.head()


#Bovenstaande code selecteert een map in path, en zet alle csv bestanden in een dataframe genaamd df, https://sparkbyexamples.com/pandas/pandas-read-multiple-csv-files/

,Description,Transaction Date,Transaction Time,Tax Type,Transaction Type,Refund Type,Product Title,Product id,Product Type,Sku Id,...,Product ID,SKU ID,Currency of Sale,Item Price,Taxes Collected,Charged Amount,City of Buyer,State of Buyer,Postal Code of Buyer,Country of Buyer
0,GPA.3345-8622-1982-51549,"Jun 1, 2021",12:15:21 PM PDT,NaN,Charge,NaN,Pathfinder DM tools (Complete Reference for Pa...,com.vansteinengroentjes.apps.ddpf,1,premium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GPA.3345-8622-1982-51549,"Jun 1, 2021",12:15:21 PM PDT,NaN,Google fee,NaN,Pathfinder DM tools (Complete Reference for Pa...,com.vansteinengroentjes.apps.ddpf,1,premium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GPA.3309-9945-4854-73987,"Jun 1, 2021",12:15:56 PM PDT,NaN,Charge,NaN,Character Manager (Complete Reference for Path...,com.vansteinengroentjes.apps.ddpf,1,unlockcharactermanager,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GPA.3309-9945-4854-73987,"Jun 1, 2021",12:15:56 PM PDT,NaN,Google fee,NaN,Character Manager (Complete Reference for Path...,com.vansteinengroentjes.apps.ddpf,1,unlockcharactermanager,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GPA.3370-7096-7934-01916,"Jun 1, 2021",5:38:09 PM PDT,NaN,Charge,NaN,Character Manager (Complete Reference for DnD 5),com.vansteinengroentjes.apps.ddfive,1,unlockcharactermanager,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
df = df[["Product id", "Sku Id", "Product Title", "Amount (Merchant Currency)", "Transaction Type"]]
#selecteert de rijen die van belang zijn (zie opdracht document)


df = df.loc[(df['Transaction Type'] == 'Charge')]
#selecteert specfieke rijen met de waarden die overeenkomen hier boven (zie opdracht document)



In [68]:
df.isna().sum()/len(df)*100

#laat het percentage NaN values zijn.


Product id                    0.000000
Sku Id                        2.257182
Product Title                 0.000000
Amount (Merchant Currency)    0.000000
Transaction Type              0.000000
dtype: float64

In [69]:
df = df.dropna()

#dropt alle rijen waar NaN in voorkomt, https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html



In [70]:
df.isna().sum()/len(df)*100

#laat zien dat er nu geen 1 rij NaN type heeft. 


Product id                    0.0
Sku Id                        0.0
Product Title                 0.0
Amount (Merchant Currency)    0.0
Transaction Type              0.0
dtype: float64

In [71]:

#vertaalt de transaction date naar leesbaar data voor pandas dataframe.
df.head()


,Product id,Sku Id,Product Title,Amount (Merchant Currency),Transaction Type
0,com.vansteinengroentjes.apps.ddpf,premium,Pathfinder DM tools (Complete Reference for Pa...,2.04,Charge
2,com.vansteinengroentjes.apps.ddpf,unlockcharactermanager,Character Manager (Complete Reference for Path...,1.63,Charge
4,com.vansteinengroentjes.apps.ddfive,unlockcharactermanager,Character Manager (Complete Reference for DnD 5),4.49,Charge
6,com.vansteinengroentjes.apps.ddfive,premium,DM Tools (Complete Reference for DnD 5),2.86,Charge
8,com.vansteinengroentjes.apps.ddfive,unlockcharactermanager,Character Manager (Complete Reference for DnD 5),4.51,Charge


In [78]:
# Determine where the visualization will be rendered
# output_file('filename.html')  # Render to static HTML, or 
output_notebook()  # Render inline in a Jupyter Notebook

#selecteert de rijen transaction date en amount
# amount_data = df_sales["Amount (Merchant Currency)"]


#maakt cds bestanden voor sales per dag
# date_data_cds = ColumnDataSource(date_amount_data.groupby('Transaction Date')['Amount (Merchant Currency)'].sum().to_frame().reset_index(), data=dict(date=dates, close=AAPL['adj_close']))
df_grouped_sku_id = df.groupby(['Sku Id'])['Amount (Merchant Currency)'].sum().to_frame().reset_index()
df_grouped_sku_id.columns = ['sku_id', 'amount']
sku_id_volume_cds = ColumnDataSource(df_grouped_sku_id)
# amount_data_cds = ColumnDataSource(amount_data)
df_grouped_sku_id.head()

Loading BokehJS ...

,sku_id,amount
0,charpf2,126.41
1,com.vansteinengroentjes.apps.whisky.adverts,4.29
2,dmtoolspf2,87.17
3,premium,1587.84
4,unlockcharactermanager,3328.09


In [91]:
# Set up the figure(s)
# Create and configure the figure
from bokeh.models import HoverTool
#add hovertool
hover = HoverTool(tooltips = [ ('Sku ID','@sku_id'),('Total Sum of Sales', '@amount')])
labels = df_grouped_sku_id['sku_id'].tolist()
amount = df_grouped_sku_id['amount'].tolist()
print(labels)

fig = figure(plot_height=800, plot_width=1500, tools=['xpan', 'reset', 'save', 'wheel_zoom', hover],
            title='Sales volume (EUR) per SKU ID in the second half of 2021', x_axis_type='auto',
            x_axis_label='Sku ID', y_axis_label='Amount', x_range=labels)
# Connect to and draw the data
# Render the race as step lines
#maak vbar fig

fig.vbar(x='sku_id', top='amount', source=sku_id_volume_cds, width=0.5)


['charpf2', 'com.vansteinengroentjes.apps.whisky.adverts', 'dmtoolspf2', 'premium', 'unlockcharactermanager']


In [74]:
# xdate1 = datetime.date(2018, 6, 1)
# xdate2 = datetime.date(2018, 6, 3)
# print(xdate1,xdate2)

# p = figure(height=300, width=800, tools="xpan", toolbar_location=None,
#            x_axis_type="datetime", x_axis_location="above",
#            background_fill_color="#efefef", x_range=(0,5))

# p.line(x='Transaction Date', y='Amount (Merchant Currency)', source=date_data_cds)


# select = figure(title="Drag the middle and edges of the selection box to change the range above",
#                 height=130, width=800, y_range=p.y_range,
#                 x_axis_type="datetime", y_axis_type=None,
#                 tools="", toolbar_location=None, background_fill_color="#efefef")

# range_tool = RangeTool(x_range=p.x_range)
# range_tool.overlay.fill_color = "navy"
# range_tool.overlay.fill_alpha = 0.2

# select.line(x= 'Transaction Date', y= 'Amount (Merchant Currency)', source=date_data_cds)
# select.ygrid.grid_line_color = None
# select.add_tools(range_tool)
# select.toolbar.active_multi = range_tool

# show(column(p, select))
# output_file('test.html')



In [75]:


# #range slider
# select = figure(title="Drag the middle and edges of the selection box to change the range above",
#                 height=130, width=800, y_range=fig.y_range,
#                 x_axis_type="datetime", y_axis_type=None,
#                 tools="", toolbar_location=None, background_fill_color="#efefef")

# range_tool = RangeTool(x_range=Range1d(0,5))
# range_tool.overlay.fill_color = "navy"
# range_tool.overlay.fill_alpha = 0.2

# select.vbar(x='Transaction Date', top='Amount (Merchant Currency)', source=date_data_cds, width=20)
# select.ygrid.grid_line_color = None
# select.add_tools(range_tool)
# select.toolbar.active_multi = range_tool

# # Organize the layout

# # Preview and save 
# show(column(fig,select))  # See what I made, and save if I like it
# output_file('test.html')
